# Sentence Classification with Huggingface-Trainer

Creators: Luca Moroni

Official Reference: https://huggingface.co/blog/sentiment-analysis-python

To face this notebook you must read and understand well the "NLP Notebook #5 - Transformers".

In this EXTRA notebook we will see the usage of Huggingface Trainer, a tested pipeline that led us to use less code and concentrate on other aspects than create the wheel again every times (e.g. training for loop, early stopping, ...).


Let's go through the code!

In [ ]:
import torch
import numpy as np
import pandas as pd
from typing import Dict
import torch
from datasets import load_dataset
from transformers import DataCollatorWithPadding

from datasets import load_dataset
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    EvalPrediction,
    Trainer,
    TrainingArguments,
    set_seed,
)

In [1]:



### Model Parameters
# we will use with Distil-BERT
language_model_name = "roberta-base"

### Training Argurments

# this GPU should be enough for this task to handle 32 samples per batch
batch_size = 8

# optim
learning_rate = 1e-4
weight_decay = 0.001 # we could use e.g. 0.01 in case of very low and very high amount of data for regularization

# training
epochs = 1
device = "cuda" if torch.cuda.is_available() else "cpu"


set_seed(42)

In this example we will use **Sentiment Tree Bank corpus (SST2)**, one of the main benchmark for binary sentiment analysis task.

The dataset if freely available through Huggingface Datasets.

In [2]:
# load our dataset
sst2_dataset = load_dataset("stanfordnlp/sst2")

In [3]:
## Let's see an example...
print(f"Sentence: {sst2_dataset['train']['sentence'][42]}")
print(f"Sentiment: {'Positive' if sst2_dataset['train']['label'][42] == 1 else 'Negative'}")

Sentence: as they come , already having been recycled more times than i 'd care to count 
Sentiment: Negative


In [4]:
## The structure of the huggingface dataset.
## Here the test set cannot be used since is a blind test set, so there aren't the gold labels.
sst2_dataset

DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 872
    })
    test: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 1821
    })
})

### Metric Definition

Looking only at cross entropy loss cannot allow us to understand effectivelly the real capabilities of our NLP model. So let's define a standard method to compute:

- **Accuracy** metric
- **F1** metric

In [5]:
from datasets import load_metric

# Metrics

def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")

   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}

## The Model

We will rely on the Huggingface **AutoModelForSequenceClassification** class from huggingface repository. This is a wrapper for encoder-only models, that allow us to simply create a model suitable to solve a classification task over textual sentences.

### Sentence Classification

In the previous notebook you saw how to train a token level classifier to solve NER task, learning a MLP over each tokens of the input sentence. In the setting of sentence level classification the standard approaches apply a MLP over the [CLS] token's embedding of the last encoder layer. This is because the [CLS] token contains a lot of information about the **semantic and syntactitc** structure of the input sentence.

![alt text](https://jalammar.github.io/images/bert-classifier.png "Sentence Classification")

In [6]:
## Initialize the model
model = AutoModelForSequenceClassification.from_pretrained(language_model_name,
                                                                   ignore_mismatched_sizes=True,
                                                                   output_attentions=False, output_hidden_states=False,
                                                                   num_labels=2) # number of the classes

tokenizer = AutoTokenizer.from_pretrained(language_model_name)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding=True, truncation=True)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# Tokenize the dataset ...
print("Tokenize the dataset ...")
tokenized_datasets_sst2 = sst2_dataset.map(tokenize_function, batched=True)

Tokenize the dataset ...


Map:   0%|          | 0/872 [00:00<?, ? examples/s]

## Model Training

To train a transformer model you can rely on the **Trainer** class of Huggingface (https://huggingface.co/docs/transformers/main_classes/trainer).

The Trainer class allows you to save many lines of code, and makes your code much more readable.

To initialize the Trainer class you have to define a **TrainerArguments** object.

In [8]:
training_args = TrainingArguments(
    output_dir="training_dir",                    # output directory [Mandatory]
    num_train_epochs=epochs,                      # total number of training epochs
    per_device_train_batch_size=batch_size,       # batch size per device during training
    warmup_steps=500,                             # number of warmup steps for learning rate scheduler
    weight_decay=weight_decay,                    # strength of weight decay
    save_strategy="no",
    learning_rate=learning_rate                   # learning rate
)

In [9]:
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_datasets_sst2["train"],
   eval_dataset=tokenized_datasets_sst2["validation"],
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [10]:
# Let's Train ...
trainer.train()

  0%|          | 0/8419 [00:00<?, ?it/s]

{'loss': 0.5944, 'grad_norm': 2.0969250202178955, 'learning_rate': 0.0001, 'epoch': 0.06}
{'loss': 0.7004, 'grad_norm': 1.1538499593734741, 'learning_rate': 9.368607147367092e-05, 'epoch': 0.12}
{'loss': 0.7024, 'grad_norm': 1.4970831871032715, 'learning_rate': 8.737214294734185e-05, 'epoch': 0.18}
{'loss': 0.6955, 'grad_norm': 1.297337532043457, 'learning_rate': 8.105821442101276e-05, 'epoch': 0.24}
{'loss': 0.6957, 'grad_norm': 0.6537973284721375, 'learning_rate': 7.474428589468367e-05, 'epoch': 0.3}
{'loss': 0.6904, 'grad_norm': 1.0907007455825806, 'learning_rate': 6.843035736835459e-05, 'epoch': 0.36}
{'loss': 0.6891, 'grad_norm': 0.5343977212905884, 'learning_rate': 6.211642884202552e-05, 'epoch': 0.42}
{'loss': 0.6915, 'grad_norm': 1.2975486516952515, 'learning_rate': 5.580250031569643e-05, 'epoch': 0.48}
{'loss': 0.6916, 'grad_norm': 3.150383949279785, 'learning_rate': 4.948857178936735e-05, 'epoch': 0.53}
{'loss': 0.6841, 'grad_norm': 2.475999355316162, 'learning_rate': 4.31746

TrainOutput(global_step=8419, training_loss=0.6855314222816787, metrics={'train_runtime': 2560.9438, 'train_samples_per_second': 26.299, 'train_steps_per_second': 3.287, 'total_flos': 2206919518482660.0, 'train_loss': 0.6855314222816787, 'epoch': 1.0})

In [12]:
# Evaluate the model ...
trainer.evaluate()

/home/dario/repos/MNLP_HW3/.mnlp/lib/python3.12/site-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/home/dario/repos/MNLP_HW3/.mnlp/lib/python3.12/site-packages/datasets/load.py:759: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mand

{'eval_loss': 0.7017666697502136,
 'eval_accuracy': 0.5091743119266054,
 'eval_f1': 0.6747720364741642,
 'eval_runtime': 11.3533,
 'eval_samples_per_second': 76.806,
 'eval_steps_per_second': 9.601,
 'epoch': 1.0}